In [2]:
from PIL import Image 
import numpy

import json

def color_simplify(img, track_name):
    
    img_arr = numpy.array(img)
    w, h = img.size

    param = 128
    interval = [0, 64, 128, 192, 256]
    
    for row in range(0, h):
        for col in range(0, w):
            cell = img_arr[row][col]

            for i in range(0, 3):
                if cell[i] < interval[1]:
                    img_arr[row][col][i] = interval[0]
                else:
                    img_arr[row][col][i] = (int(cell[i] / param) + 1) * param - 1
                    
    img = Image.fromarray(numpy.uint8(img_arr) , 'RGB')
    img.save(track_name + '_simplified.png')
    turn_into_grayscale(img, track_name)

def turn_into_grayscale(img, track_name):
    img_arr = numpy.array(img)
    w, h = img.size

    green_img = img_arr

    mat = [127, 255, 127]
    
    for row in range(0, h):
        for col in range(0, w):
            cell = img_arr[row][col]
            if (cell==mat).all():
                green_img[row][col] = [255, 255, 255]
            else:
                green_img[row][col] = [0, 0, 0]

    img = Image.fromarray(numpy.uint8(green_img) , 'RGB')
    img.save(track_name + '_grayscale.png')
    output_track_data(img, track_name)
    

def output_track_data(img, track_name):
    
    img = Image.open(track_name + '_grayscale.png').convert('L')
    img_arr = numpy.array(img)
    w, h = img.size
    
    # 1. check left and right
    left_line = -1
    right_line = -1

    for col in range(0, w):
        count = 0
        for row in range(0, h):
            if img_arr[row][col] != 0:
                count += 1
        if count > 10:
            if left_line == -1:
                left_line = col

            else:
                right_line = col

    # 2. find bottom
    bottom_line = -1

    for row in range(0, h):
        count = 0
        for col in range(0, w):
            if img_arr[row][col] != 0:
                count += 1
        if count > 10:
            bottom_line = row

    # 3. find zero_line
    zero_line = -1
    for row in range(0, h):
        if img_arr[row][left_line + 1] != 0:
            zero_line = row
            break

    print(zero_line)

    print("left & right: ", left_line, right_line)
    print("zero & bottom: ", zero_line, bottom_line)

    # 4. count ordinate and save the data

    full_distance = 1894.3
    meter_per = 2 / (bottom_line - zero_line)
    distance_per = 1894.3 / (right_line - left_line)
    print("distance_per: ", distance_per)
    print("2m: ", bottom_line - zero_line)
    print("meter_per: ", meter_per)

    pos = {}
    max_height = 0
    min_height = 0
    for distance in range(left_line, right_line + 1):
        for row in range(0, h):
            if img_arr[row][distance] != 0:
                
                h_pos = int((zero_line - row) * meter_per * 100)
                
                if h_pos > max_height:
                    max_height = h_pos
                if h_pos < min_height:
                    min_height = h_pos
                
                pos[int((distance - left_line) * distance_per * 100)] = h_pos
                break

    data = {}
    data["distance"] = full_distance
    data["max_height"] = max_height
    data["min_height"] = min_height
    data["distance_unit"] = "m"
    data["height_unit"] = "0.01m"
    data["pos_unit"] = "0.01m"
    data["pos"] = pos

    with open(track_name +'.json', 'w') as outfile:
        json.dump(data, outfile)
    
    fill_up_the_track(img, track_name)
    
def fill_up_the_track(img, track_name):

    with open(track_name + '.json') as outfile:
        track_data = json.load(outfile)

    distance = track_data["distance"]
    pos = track_data["pos"]

    height = []
    for i in range(0, int(distance * 100) + 1):
        height.append({"height": -10000, "slope": -10000})

    for x in pos:
        height[int(x)]["height"] = pos[x]
#         print(x)

    for x in pos:
        start_point = int(x)

        i = 1
        if start_point + i >= len(height):
            break
        while height[start_point + i]["height"] == -10000:
            i += 1
        end_point = start_point + i

        height_per = (height[end_point]["height"] - height[start_point]["height"]) / (end_point - start_point)
        height_per = int(height_per * 100000) / 100000
        #         print(height_per)
        height[start_point]["slope"] = height_per

        for i in range(1, end_point - start_point + 1):
            height[start_point + i]["height"] = int(1000 * (height[start_point]["height"] + i * height_per)) / 1000
            height[start_point + i]["slope"] = height_per
#             print(height[start_point + i]["height"], height[start_point + i]["slope"])
            
            
            
            
    height_data = {}

    i = 0
    while i <= distance * 10:
        height_data[str(i / 10)] = {"height": height[int(i * 10)]["height"] / 100, "slope": height[int(i * 10)]["slope"]}
#         print(height_data[str(i/10)])
        i += 1

#     print(height_data)



    with open(track_name + '_track.json', 'w') as outfile:
        json.dump(height_data, outfile)

    
track_name = "Kyoto_inside"
img = Image.open(track_name + ".png").convert('RGB')


color_simplify(img, track_name)


99
left & right:  38 532
zero & bottom:  99 124
distance_per:  3.8346153846153843
2m:  25
meter_per:  0.08


In [61]:
from PIL import Image 
import numpy

import json

with open('Kyoto_outside_track.json') as outfile:
    track_data = json.load(outfile)

m = 100000
M = -1
for pos in track_data.keys():
#     print(pos, track_data[pos]["height"], track_data[pos]["slope"])
#     print(track_data[pos]["slope"] * 200 / 1)
    if track_data[pos]["slope"] > M:
        M = track_data[pos]["slope"]
    if track_data[pos]["slope"] < m:
        m = track_data[pos]["slope"]
#     print()
print(M, m)

for i in range(1, 12):
    print(i * 100, int(m * 200 / (i * 18.7) * 1000) / 10, "%")
print(M * 200 / 1, m * 200 / 1)
# print(track_data)

0.04178 -0.04177
100 -44.6 %
200 -22.3 %
300 -14.8 %
400 -11.1 %
500 -8.9 %
600 -7.4 %
700 -6.3 %
800 -5.5 %
900 -4.9 %
1000 -4.4 %
1100 -4.0 %
8.356 -8.354000000000001
